In [15]:
!az login --tenant fdpo.onmicrosoft.com

In [12]:
## Import the pip dependencies
import pandas as pd
from datetime import datetime

import sys
# !{sys.executable} -m pip install azure-ai-evaluation
!{sys.executable} -m pip install azure-ai-evaluation[remote]

In [2]:
dataset = pd.read_csv("./slm_perf_output.csv")

sample = dataset.iloc[0]
print(sample)

Prompt      [{'role': 'system', 'content': 'You are a help...
Actual      {'tool_uses': [{'recipient_name': 'functions.s...
Expected    {'tool_uses': [{'recipient_name': 'functions.s...
Match                                                      No
Name: 0, dtype: object


In [61]:
import yaml

from azure.ai.evaluation import BleuScoreEvaluator, GleuScoreEvaluator, MeteorScoreEvaluator, RougeScoreEvaluator, RougeType

bleu = BleuScoreEvaluator()
glue = GleuScoreEvaluator()
meteor = MeteorScoreEvaluator(alpha = 0.9, beta = 3.0, gamma = 0.5)
rouge = RougeScoreEvaluator(rouge_type=RougeType.ROUGE_L)


ModuleNotFoundError: No module named 'custom_evaluators'

In [19]:
with open('./llama-fc_config.yaml') as f:
    d = yaml.load(f, Loader=yaml.FullLoader)
    
AZURE_SUBSCRIPTION_ID = d['config']['AZURE_SUBSCRIPTION_ID']
AZURE_RESOURCE_GROUP = d['config']['AZURE_RESOURCE_GROUP']
AZURE_WORKSPACE = d['config']['AZURE_WORKSPACE']
AZURE_DATA_NAME = d['config']['AZURE_SFT_DATA_NAME']    
DATA_DIR = d['config']['SFT_DATA_DIR']
CLOUD_DIR = d['config']['CLOUD_DIR']
HF_MODEL_NAME_OR_PATH = d['config']['HF_MODEL_NAME_OR_PATH']


In [5]:
df = pd.read_csv("./slm_perf_output.csv")
df.rename(columns = {"Actual" : "ground_truth", "Expected" : "response"}, inplace = True)
df.to_json("./eval_baseline_data.jsonl", orient = "records", lines = True)

In [ ]:
# <mySubscriptionID>: Subscription ID of the Azure AI Studio hub's linked storage account (available in Azure AI hub resource view in Azure Portal).
# <myResourceGroupName>: Resource group of the Azure AI Studio hub's linked storage account.
# <user-id>: User object ID for role assignment (retrieve with "az ad user show" command).

! az role assignment create --role "Storage Blob Data Contributor" --scope /subscriptions/8cebb108-a4d5-402b-a0c4-f7556126277f/resourceGroups/openairesourcegroup_priya --assignee-principal-type User --assignee-object-id "8b6b15cc-9937-4208-8173-97ea0b9e1a13"

In [21]:
result = evaluate(
    data="./eval_baseline_data.jsonl",
    evaluators={
        "bleu": bleu,
        "gleu": glue,
        "meteor": meteor,
        "rouge" : rouge
    },
    # Optionally provide your AI Studio project information to track your evaluation results in your Azure AI Studio project
    azure_ai_project={
        "subscription_id": "8cebb108-a4d5-402b-a0c4-f7556126277f",
        "resource_group_name": "openAIResourceGroup_priya",
        "project_name": "priyakedia-1214",
    },
)

Prompt flow service has started...
Prompt flow service has started...
Prompt flow service has started...
You can view the traces in local from http://127.0.0.1:23333/v1.0/ui/traces/?#run=azure_ai_evaluation_evaluators_bleu_bleu_asyncbleuscoreevaluator_ybi6iz8s_20250102_084844_994039
You can view the traces in local from http://127.0.0.1:23333/v1.0/ui/traces/?#run=azure_ai_evaluation_evaluators_gleu_gleu_asyncgleuscoreevaluator_76w_hv1i_20250102_084845_003956
You can view the traces in local from http://127.0.0.1:23333/v1.0/ui/traces/?#run=azure_ai_evaluation_evaluators_meteor_meteor_asyncmeteorscoreevaluator_ne3uinm9_20250102_084844_999574


In [57]:
import pandas as pd
df = pd.DataFrame(result["rows"])

# df

# sample_df = df.loc[df["inputs.ground_truth"].str.startswith("{'tool_uses'}")]
# sample_df = df.loc[df["inputs.ground_truth"].str.startswith("{'tool_uses'}")]  

# sample_df

index = 19

# df.iloc[index]["inputs.ground_truth"]
df.iloc[index]["inputs.response"]

# print("\n", df.iloc[0]["inputs.response"])

"{'tool_uses': [{'recipient_name': 'functions.calculate_age', 'parameters': {'birthdate': '1990-05-15'}}]}"

In [11]:
from azure.ai.evaluation import GleuScoreEvaluator

gleu = GleuScoreEvaluator()